<a href="https://colab.research.google.com/github/Duru4920/Eurostat/blob/main/Eurostat_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
!pip install linearmodels

In [26]:
from google.colab import files
import pandas as pd
import numpy as np
import io

uploaded = files.upload()
df = pd.read_csv(io.BytesIO(list(uploaded.values())[0]))
df.head()

Saving macro_panel_updated.csv to macro_panel_updated (3).csv


,country,year,quarter,npl_ratio,euribor_3m,gdp_growth,hicp_yoy,unemployment,inflation_yoy
0,AT,2000,1,NaN,3.542300,4.600445,0.0,3.8,1.733333
1,AT,2000,2,NaN,4.263000,4.600445,0.0,3.8,1.933333
2,AT,2000,3,NaN,4.737600,4.600445,0.0,3.8,2.033333
3,AT,2000,4,NaN,5.024167,4.600445,0.0,3.8,2.100000
4,BE,2000,1,NaN,3.542300,5.806174,0.0,7.1,1.633333


In [27]:
# Standardize
df.columns = df.columns.str.strip().str.lower()

# DROP the wrong inflation column
if "hicp_yoy" in df.columns:
    df = df.drop(columns=["hicp_yoy"])

# Keep only the correct inflation column
print(df.columns.tolist())

['country', 'year', 'quarter', 'npl_ratio', 'euribor_3m', 'gdp_growth', 'unemployment', 'inflation_yoy']


In [29]:
df["time_id"] = df["year"] * 4 + df["quarter"]

num_cols = ["year","quarter","time_id","npl_ratio",
            "gdp_growth","unemployment","inflation_yoy","euribor_3m"]

for c in num_cols:
    df[c] = pd.to_numeric(df[c], errors="coerce")

In [30]:
df = df.dropna(subset=["npl_ratio"])
df = df.sort_values(["country","time_id"]).reset_index(drop=True)

df.head()

,country,year,quarter,npl_ratio,euribor_3m,gdp_growth,unemployment,inflation_yoy,time_id
0,AT,2006,1,2.739429,2.611567,5.380802,5.7,1.433333,8025
1,AT,2006,2,2.739429,2.889500,5.380802,5.7,2.033333,8026
2,AT,2006,3,2.739429,3.221367,5.380802,5.7,1.800000,8027
3,AT,2006,4,2.739429,3.594467,5.380802,5.7,1.433333,8028
4,AT,2007,1,2.242467,3.820333,6.119790,5.3,1.766667,8029


In [31]:
df_panel = df.set_index(["country","time_id"]).sort_index()
df_panel.head()

year  quarter  npl_ratio  euribor_3m  gdp_growth  \
country time_id                                                     
AT      8025     2006        1   2.739429    2.611567    5.380802   
        8026     2006        2   2.739429    2.889500    5.380802   
        8027     2006        3   2.739429    3.221367    5.380802   
        8028     2006        4   2.739429    3.594467    5.380802   
        8029     2007        1   2.242467    3.820333    6.119790   

                 unemployment  inflation_yoy  
country time_id                               
AT      8025              5.7       1.433333  
        8026              5.7       2.033333  
        8027              5.7       1.800000  
        8028              5.7       1.433333  
        8029              5.3       1.766667

In [32]:
vars_analysis = ["npl_ratio","gdp_growth","unemployment","inflation_yoy","euribor_3m"]
print(df_panel[vars_analysis].describe())

         npl_ratio   gdp_growth  unemployment  inflation_yoy   euribor_3m
count  1288.000000  1288.000000   1288.000000    1288.000000  1288.000000
mean      6.262905     4.332987      8.822360       2.477666     0.964973
std       8.042270     5.759178      4.539123       3.191348     1.636366
min       0.145588   -21.188764      3.100000      -3.866667    -0.566400
25%       1.992508     1.906078      5.900000       0.633333    -0.319500
50%       3.351983     4.012819      7.350000       1.816667     0.223500
75%       6.375134     7.305004     10.300000       3.166667     1.772133
max      47.747846    35.737719     27.800000      24.166667     4.981800


In [33]:
print(df_panel[vars_analysis].corr())

               npl_ratio  gdp_growth  unemployment  inflation_yoy  euribor_3m
npl_ratio       1.000000   -0.235386      0.624301      -0.261648   -0.199448
gdp_growth     -0.235386    1.000000     -0.340156       0.384545    0.099805
unemployment    0.624301   -0.340156      1.000000      -0.265070   -0.192088
inflation_yoy  -0.261648    0.384545     -0.265070       1.000000    0.292140
euribor_3m     -0.199448    0.099805     -0.192088       0.292140    1.000000


In [34]:
from linearmodels.panel import PanelOLS
import statsmodels.api as sm

y = df_panel["npl_ratio"]
X = df_panel[["gdp_growth","unemployment","inflation_yoy","euribor_3m"]]

fe = PanelOLS(y, X, entity_effects=True, check_rank=False)
fe_res = fe.fit(cov_type="clustered", cluster_entity=True)
print(fe_res)

                          PanelOLS Estimation Summary                           
Dep. Variable:              npl_ratio   R-squared:                        0.5060
Estimator:                   PanelOLS   R-squared (Between):              0.1981
No. Observations:                1288   R-squared (Within):               0.5060
Date:                Tue, Nov 25 2025   R-squared (Overall):              0.3497
Time:                        22:12:57   Log-likelihood                   -3682.2
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      323.96
Entities:                          19   P-value                           0.0000
Avg Obs:                       67.789   Distribution:                  F(4,1265)
Min Obs:                       12.000                                           
Max Obs:                       80.000   F-statistic (robust):             8.2919
                            

In [35]:
from linearmodels.panel import RandomEffects

X_re = sm.add_constant(X)
re = RandomEffects(y, X_re)
re_res = re.fit()
print(re_res)

                        RandomEffects Estimation Summary                        
Dep. Variable:              npl_ratio   R-squared:                        0.5028
Estimator:              RandomEffects   R-squared (Between):              0.1520
No. Observations:                1288   R-squared (Within):               0.5060
Date:                Tue, Nov 25 2025   R-squared (Overall):              0.3780
Time:                        22:13:17   Log-likelihood                   -3691.9
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      324.35
Entities:                          19   P-value                           0.0000
Avg Obs:                       67.789   Distribution:                  F(4,1283)
Min Obs:                       12.000                                           
Max Obs:                       80.000   F-statistic (robust):             324.54
                            

In [36]:
from scipy import stats

beta = ["gdp_growth","unemployment","inflation_yoy","euribor_3m"]

b_fe = fe_res.params[beta]
b_re = re_res.params[beta]

V_fe = fe_res.cov.loc[beta, beta]
V_re = re_res.cov.loc[beta, beta]

diff = b_fe - b_re
Vdiff = V_fe - V_re

H = float(diff.T @ np.linalg.inv(Vdiff) @ diff)
p = 1 - stats.chi2.cdf(H, len(diff))

print("Hausman chi2:", H)
print("p-value:", p)

Hausman chi2: -0.03990132049242252
p-value: 1.0


In [37]:
df_panel.reset_index().to_csv("macro_clean_final.csv", index=False)
from google.colab import files
files.download("macro_clean_final.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>